In [11]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from math import cos,sin
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import interact
import plotly.express as px
import pickle

import ipywidgets
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox, Output, HTML, Dropdown, Button, Layout, Label
from IPython.display import display, clear_output

import requests
import json

import gc

In [3]:
with open("city_monthly_nn_model.pkl","rb") as f:
    lm = pickle.load(f)

In [4]:
df = pd.read_csv("cities_minustemps_expanded.csv",compression="gzip")

In [ ]:
sc = StandardScaler()
sc.fit_transform(X_train)


In [5]:
model_df = pd.DataFrame()
for city in df.City.unique():
    mydf = df.query("City == '"+city+"'")
    mydf['TempMinus1'] = mydf.shift(12)['AverageTemperature']
    mydf['TempMinus2'] = mydf.shift(24)['AverageTemperature']
    model_df = pd.concat([model_df,mydf])
model_df

,Unnamed: 0,level_0,index,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,x,y,z,Month,Year,TempMinus1,TempMinus2
0,0,0.0,0.0,1743-11-01,6.068,1.737,Århus,Denmark,57.05,10.33,-0.541522,-0.689770,0.480594,11,1743,NaN,NaN
1,1,1.0,1.0,1743-12-01,NaN,NaN,Århus,Denmark,57.05,10.33,-0.541522,-0.689770,0.480594,12,1743,NaN,NaN
2,2,2.0,2.0,1744-01-01,NaN,NaN,Århus,Denmark,57.05,10.33,-0.541522,-0.689770,0.480594,1,1744,NaN,NaN
3,3,3.0,3.0,1744-02-01,NaN,NaN,Århus,Denmark,57.05,10.33,-0.541522,-0.689770,0.480594,2,1744,NaN,NaN
4,4,4.0,4.0,1744-03-01,NaN,NaN,Århus,Denmark,57.05,10.33,-0.541522,-0.689770,0.480594,3,1744,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10512847,550,NaN,NaN,2059-08-31,NaN,NaN,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,8,2059,NaN,NaN
10512848,551,NaN,NaN,2059-09-30,NaN,NaN,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,9,2059,NaN,NaN
10512849,552,NaN,NaN,2059-10-31,NaN,NaN,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,10,2059,NaN,NaN
10512850,553,NaN,NaN,2059-11-30,NaN,NaN,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,11,2059,NaN,NaN


In [12]:
gc.collect()

18

In [13]:
model_df.to_csv('cities_minustemps_expanded_x.csv',compression="gzip")

In [16]:
model_df.query("City=='Zwolle'").dropna().tail(15)

,index,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,x,y,z,Month,Year,TempMinus1,TempMinus2
3223,10512281,2012-06-01,14.702,0.342,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,6,2012,16.273,16.213
3224,10512282,2012-07-01,17.329,0.287,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,7,2012,16.020,19.775
3225,10512283,2012-08-01,18.588,0.224,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,8,2012,16.975,16.912
3226,10512284,2012-09-01,14.333,0.207,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,9,2012,15.788,13.661
3227,10512285,2012-10-01,10.358,0.208,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,10,2012,11.337,10.279
3228,10512286,2012-11-01,6.469,0.189,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,11,2012,6.990,5.385
3229,10512287,2012-12-01,4.303,0.341,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,12,2012,5.722,-1.778
3230,10512288,2013-01-01,1.479,0.217,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,1,2013,4.348,2.934
3231,10512289,2013-02-01,1.559,0.304,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,2,2013,0.376,4.210
3232,10512290,2013-03-01,2.253,0.267,Zwolle,Netherlands,52.24,5.26,-0.204533,0.335398,0.919605,3,2013,7.863,5.917


In [6]:
x_columns = ['Year','x','y','z','TempMinus1','TempMinus2', \
             'January','February','March','April','May','June','July','August','September','October','November','December']